In [ ]:
import sys
import os

# Determine the absolute path to the src directory
src_path = os.path.abspath(os.path.join(os.getcwd(), 'src'))

# Add src_path to sys.path if it's not already present
if src_path not in sys.path:
    sys.path.insert(0, src_path)

In [ ]:
import pandas as pd
from tabpfn_extensions import TabPFNRegressor, TabPFNClassifier
from tabpfn_extensions.embedding import TabPFNEmbedding

import argparse
import os
import numpy as np
import random
from tqdm import tqdm

from src.data_utils import load_data_for_tabPFN
from src.data_constants import targets

# Set the random seed for reproducibility
s = 42
np.random.seed(s)
random.seed(s)

# load the dataset
f = 'data/processed/vitals_yeo-johnson_test_data.csv'
if f.endswith(".csv"):
    data, ids = load_data_for_tabPFN(f)
    print(f"Loaded data from {f}")

# fit the model and extract the embeddings
y = data[targets]
x = data.drop(columns=targets)
vecs = []
print(x.columns)
t = targets[0]


In [ ]:
from tabpfn.config import ModelInterfaceConfig

In [ ]:
config = ModelInterfaceConfig(
    FEATURE_SHIFT_METHOD = None,
    CLASS_SHIFT_METHOD = None
)

In [ ]:
reg = TabPFNRegressor(random_state=s,  
                      categorical_features_indices=[0,1,3,5,6,7,13,25,26,27],
                      inference_config=config,
                      n_estimators = 1
                     )
reg.feature_names_in_= x.columns

In [ ]:
from tabpfn.utils import validate_X_predict, _fix_dtypes

In [ ]:
x.dtypes

In [ ]:
reg.fit(x, y[t])

In [ ]:
x = validate_X_predict(x, reg)

In [ ]:
x = _fix_dtypes(x, cat_indices=reg.categorical_features_indices)

In [ ]:
from sklearn.base import check_is_fitted, is_classifier
print(check_is_fitted(reg))

In [ ]:
tran = reg.executor_.preprocessor.transform(x)

In [ ]:
reg.executor_.preprocessors

In [ ]:
reg.executor_.ensemble_configs

In [ ]:
def iter_outputs(
    self,
    X: np.ndarray,
    *,
    device: torch.device,
    autocast: bool,
    only_return_standard_out: bool = True,
) -> Iterator[tuple[torch.Tensor | dict, EnsembleConfig]]:
    for preprocessor, X_train, y_train, config, cat_ix in zip(
        self.preprocessors,
        self.X_trains,
        self.y_trains,
        self.ensemble_configs,
        self.cat_ixs,
    ):
        X_train = torch.as_tensor(X_train, dtype=torch.float32, device=device)  # noqa: PLW2901

        X_test = preprocessor.transform(X).X
        X_test = torch.as_tensor(X_test, dtype=torch.float32, device=device)

        X_full = torch.cat([X_train, X_test], dim=0).unsqueeze(1)
        y_train = torch.as_tensor(y_train, dtype=torch.float32, device=device)  # noqa: PLW2901

In [ ]:
reg.executor_.ensemble_configs

In [ ]:
for i in reg.executor_.iter_outputs(x ,device = 'cpu', autocast = False):
    print(i)

In [ ]:
for name, transformer, columns in reg.preprocessor_.transformers_:
    print(f"Transformer: {name}")
    print(f"Applies to columns: {columns}")
    print(f"Transformer steps: {transformer}")
    print("-" * 40)

In [ ]:
shuffle_idx = [idx for i in [col for _, _, col in reg.preprocessor_.transformers_] for idx in i]

In [ ]:
sort_idx = np.argsort(shuffle_idx)

In [ ]:
x_in = tran[:, sort_idx]

In [ ]:
pd.DataFrame(x_in)

In [ ]:
data

In [ ]:
x

In [ ]:
pd.DataFrame(tran)